In [1]:
import veroviz as vrv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
from ipyleaflet import Map, Marker, Polyline
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
import json
import requests
import os
import polyline
import geopy.distance

In [2]:
# Setting up matplotlib to display inline in Jupyter notebooks
%matplotlib inline

In [3]:
def get_osrm_route(start, end, server='http://router.project-osrm.org'):
    coords = f"{start[1]},{start[0]};{end[1]},{end[0]}"
    url = f"{server}/route/v1/driving/{coords}?overview=full"
    try:
        response = requests.get(url).json()
        if response['code'] == 'Ok':
            encoded_polyline = response['routes'][0]['geometry']
            route = polyline.decode(encoded_polyline)
            # Modify here to directly create lat and lng in the outer dictionary
            return [{'lat': pt[0], 'lng': pt[1]} for pt in route], None
        else:
            return None, f"Error from OSRM: {response.get('message', 'No specific error message')}"
    except Exception as e:
        return None, f"Exception in getting OSRM route: {str(e)}"

In [4]:
def load_json_files(directory):
    # Map of route IDs to city names
    route_to_city = {
        "0ba52062-f98a-4750-a6ef-0a72ff35202e": "Chicago",
        "69233ee9-1ac4-4d89-b322-94fb13df54a3": "LA",
        "02f91cfa-3839-4e55-91a0-ab19c3e77683": "Seattle",
        "ed9dd222-9ce7-47f3-8cd9-d2a5d2b6a493": "Austin",
        "e9939a49-8f0d-439f-80d9-1840dc1376ca": "Boston"
    }

    data_objects = {}
    for filename in os.listdir(directory):
        if filename.endswith('.json'):  # Ensure it's a JSON file
            # Extract the route ID from the filename
            route_id = filename.split('_')[1]
            if route_id in route_to_city:
                file_path = os.path.join(directory, filename)
                with open(file_path, 'r') as file:
                    file_data = json.load(file)
                    city_name = route_to_city[route_id]
                    # Combine all events under each city name
                    if city_name not in data_objects:
                        data_objects[city_name] = []
                    data_objects[city_name].extend(file_data)  # Append events directly

    return data_objects

# Usage example with paths to directories
vehicle_only_events_directory_path = '../algorithm/amazon_vehicle_events'
vehicle_only_paths_directory_path = '../algorithm/amazon_vehicle_paths'

vehicle_only_events = load_json_files(vehicle_only_events_directory_path)
vehicle_only_paths = load_json_files(vehicle_only_paths_directory_path)


In [5]:
def process_single_city_events(events):
    """Process the event data for a single city and return a DataFrame with structured data."""
    # Normalize data and create DataFrame for each event list independently
    df = pd.DataFrame(events)

    # Calculate and handle each attribute, ensuring all entries are processed independently
    processed_df = pd.DataFrame({
        'id': df['id'],
        'lat': df['coordinates'].apply(lambda x: x['lat']),
        'lon': df['coordinates'].apply(lambda x: x['lng']),
        'elevMeters': 0,
        'objectID': df['id'],
        'startTimeSec': df['time'],
        'nodeName': df.apply(lambda x: f"Node {x['id']}", axis=1),
        'nodeType': 'Waypoint',
        'popupText': df.apply(lambda x: f"{x['id']} Event: {x['event_type']}<br>Time: {x['time']}", axis=1),
        'leafletIconPrefix': 'fa',
        'leafletIconType': df['id'].apply(lambda x: 'flag' if x % 2 == 0 else 'circle'),
        'leafletColor': df['id'].apply(lambda x: 'green' if x % 2 == 0 else 'blue'),
        'leafletIconText': df['id'].apply(str),
        'cesiumIconType': 'pin',
        'cesiumColor': df['id'].apply(lambda x: 'red' if x % 2 == 0 else 'yellow'),
        'cesiumIconText': df['id'].apply(str),
        'altMeters': 0,
        "stopID": df['stop_id']
    })

    return processed_df

# Dataframe to hold processed DataFrames for each city
vehicle_only_events_dataframes = {}

# Loop over each city and its associated events data
for city, events in vehicle_only_events.items():
    vehicle_only_events_dataframes[city] = process_single_city_events(events)


In [6]:
vehicle_only_nodes = vehicle_only_events_dataframes
print(vehicle_only_nodes)

{'Boston':       id        lat        lon  elevMeters  objectID  startTimeSec  nodeName  \
0      1  42.139891 -71.494346           0         1           0.0    Node 1   
1      2  42.207436 -71.684275           0         2        2013.0    Node 2   
2      3  42.207436 -71.684275           0         3        2013.0    Node 3   
3      4  42.207436 -71.684275           0         4        2075.5    Node 4   
4      5  42.207436 -71.684275           0         5        2075.5    Node 5   
..   ...        ...        ...         ...       ...           ...       ...   
657  658  42.223803 -71.728521           0       658       22920.2  Node 658   
658  659  42.223803 -71.728521           0       659       22920.2  Node 659   
659  660  42.223803 -71.728521           0       660       22999.2  Node 660   
660  661  42.223803 -71.728521           0       661       22999.2  Node 661   
661  662  42.139891 -71.494346           0       662       24930.4  Node 662   

     nodeType               

In [7]:
vehicle_only_mp = {}
for city, df in vehicle_only_nodes.items():
    vehicle_only_mp[city] = {}
    for i, row in df.iterrows():
        vehicle_only_mp[city][row['stopID']] = i

In [8]:
vehicle_props = {
    'Austin': {'modelFile': 'veroviz/models/ub_truck.gltf', 'color': 'red'},
    'Chicago': {'modelFile': 'veroviz/models/ub_truck.gltf', 'color': 'blue'},
    'Seattle': {'modelFile': 'veroviz/models/ub_truck.gltf', 'color': 'green'},
    'Boston': {'modelFile': 'veroviz/models/ub_truck.gltf', 'color': 'yellow'},
    'LA': {'modelFile': 'veroviz/models/ub_truck.gltf', 'color': 'purple'}
}

def create_vehicle_only_rows(odID, objectID, vehicle_props, route, times):
    if not route:
        print(f"Empty route received for {objectID}.")
        return []

    rows = []
    for i in range(len(route) - 1):
        start_point = route[i]
        end_point = route[i+1]
        row = {
            'odID': odID + i,
            'objectID': objectID,
            'modelFile': vehicle_props[objectID]['modelFile'],
            'startLat': start_point['lat'],
            'startLon': start_point['lng'],
            'endLat': end_point['lat'],
            'endLon': end_point['lng'],
            'startElevMeters': 0,  # Example default value
            'endElevMeters': 0,
            'startAltMeters': 100 if objectID == 'drone' else 0,  # Example customization for drones
            'endAltMeters': 100 if objectID == 'drone' else 0,
            'startTimeSec': times[i],
            'endTimeSec': times[i+1],
            'leafletColor': vehicle_props[objectID]['color'],
            'leafletWeight': 5,
            'leafletStyle': 'solid',
            'leafletOpacity': 0.8,
            'leafletCurveType': 'straight',
            'leafletCurvature': 0,
            'arcCurveType': 'straight',
            'useArrows': True,
            'cesiumColor': vehicle_props[objectID]['color'],
            'cesiumWeight': 5,
            'cesiumStyle': 'solid',
            'cesiumOpacity': 0.8,
            'popupText': f'Route segment {i+1} for {objectID}'
        }
        rows.append(row)

    return rows


def process_vehicle_paths(vehicle_paths, vehicle_props, mp):
    all_rows = []
    odID = 1

    for objectID, path in vehicle_paths.items():
        print(f"Processing path for {objectID} with {len(path)} stops.")
        for i in range(len(path) - 1):
            start = path[i]['coordinates']
            end = path[i+1]['coordinates']
            route, error = get_osrm_route((start['lat'], start['lng']), (end['lat'], end['lng']))
            if error:
                print(f"Error processing segment {i+1} in {objectID}: {error}")
                continue  # Skip this segment due to error

            # interpolate times
            times = []
            start_time = vehicle_only_nodes[objectID].loc[mp[objectID][path[i]['stop']], 'startTimeSec']
            end_time = vehicle_only_nodes[objectID].loc[mp[objectID][path[i+1]['stop']], 'startTimeSec']
            if route:
                tot = 0
                for i in range(len(route) - 1):
                    di = geopy.distance.distance((start['lat'], start['lng']), (end['lat'], end['lng'])).km
                    tot += di
                times.append(start_time)
                for i in range(len(route) - 1):
                    di = geopy.distance.distance((start['lat'], start['lng']), (end['lat'], end['lng'])).km
                    times.append(times[-1] if tot == 0 else times[-1] + (end_time - start_time) * di / tot)

            rows = create_vehicle_only_rows(odID, objectID, vehicle_props, route, times)
            all_rows.extend(rows)
            odID += len(rows)

    return all_rows


In [9]:
def validate_route_data(routes):
    for i, route in enumerate(routes):
        if 'stop' not in route or 'coordinates' not in route or 'id' not in route:
            print(f"Data missing in route at index {i}: {route}")
        if 'lat' not in route['coordinates'] or 'lng' not in route['coordinates']:
            print(f"Coordinate data missing in route at index {i}: {route}")

# Example use:
validate_route_data(vehicle_only_paths['Boston'])

In [10]:
vehicle_only_rows = process_vehicle_paths(vehicle_only_paths, vehicle_props, vehicle_only_mp)

Processing path for Boston with 167 stops.


In [ ]:
vehicle_only_assignments = vehicle_only_rows

NameError: name 'vehicle_only_rows' is not defined

In [ ]:
vehicle_only_assignments

[{'odID': 1,
  'objectID': 'Boston',
  'modelFile': 'truck.glb',
  'startLat': 42.13989,
  'startLon': -71.4943,
  'endLat': 42.14031,
  'endLon': -71.49422,
  'startElevMeters': 0,
  'endElevMeters': 0,
  'startAltMeters': 0,
  'endAltMeters': 0,
  'startTimeSec': None,
  'endTimeSec': None,
  'leafletColor': 'yellow',
  'leafletWeight': 5,
  'leafletStyle': 'solid',
  'leafletOpacity': 0.8,
  'leafletCurveType': 'straight',
  'leafletCurvature': 0,
  'arcCurveType': 'straight',
  'useArrows': True,
  'cesiumColor': 'yellow',
  'cesiumWeight': 5,
  'cesiumStyle': 'solid',
  'cesiumOpacity': 0.8,
  'popupText': 'Route segment 1 for Boston'},
 {'odID': 2,
  'objectID': 'Boston',
  'modelFile': 'truck.glb',
  'startLat': 42.14031,
  'startLon': -71.49422,
  'endLat': 42.14044,
  'endLon': -71.49443,
  'startElevMeters': 0,
  'endElevMeters': 0,
  'startAltMeters': 0,
  'endAltMeters': 0,
  'startTimeSec': None,
  'endTimeSec': None,
  'leafletColor': 'yellow',
  'leafletWeight': 5,
  'le

In [ ]:
assignments = vehicle_only_assignments
assignments['modelScale'] = 100
assignments['modelMinPxSize'] = 75
assignments['ganttColor'] = 'darkgray'
assignments['wayname'] = None
assignments['waycategory'] = None
assignments['surface'] = None
assignments['waytype'] = None
assignments['steepness'] = None
assignments['tollway'] = None
nodes_df = pd.DataFrame()
for city, df in vehicle_only_nodes.items():
    nodes_df = pd.concat([nodes_df, df], ignore_index=True)
vrv.createCesium(assignments=assignments,
                 nodes=nodes_df,
                 cesiumDir=os.getcwd()+'/Cesium-1.116',
                 problemDir='veroviz/vehicle_only')

In [11]:
vehicle_naive_events_directory_path = '../algorithm/naive_vehicle_events'
vehicle_naive_paths_directory_path = '../algorithm/naive_vehicle_paths'
drone_naive_events_directory_path = '../algorithm/naive_drone_events'
drone_naive_paths_directory_path = '../algorithm/naive_drone_paths'


vehicle_naive_events = load_json_files(vehicle_naive_events_directory_path)
vehicle_naive_paths = load_json_files(vehicle_naive_paths_directory_path)
drone_naive_events = load_json_files(drone_naive_events_directory_path)
drone_naive_paths = load_json_files(drone_naive_paths_directory_path)

In [12]:
for key, value in drone_naive_events.items():
    print({key:value})

{'LA': [{'id': 1, 'event_type': 'DEPART', 'stop_id': 'BO', 'time': 0, 'coordinates': {'lat': 33.918699, 'lng': -118.324843}}, {'id': 2, 'event_type': 'ARRIVE', 'stop_id': 'DR', 'time': 2288.5, 'coordinates': {'lat': 33.762769, 'lng': -118.375247}}, {'id': 3, 'event_type': 'DEPART', 'stop_id': 'DR', 'time': 2356.0, 'coordinates': {'lat': 33.762769, 'lng': -118.375247}}, {'id': 4, 'event_type': 'ARRIVE', 'stop_id': 'WR', 'time': 2369.7, 'coordinates': {'lat': 33.763597, 'lng': -118.375584}}, {'id': 5, 'event_type': 'DEPART', 'stop_id': 'WR', 'time': 2475.7, 'coordinates': {'lat': 33.763597, 'lng': -118.375584}}, {'id': 6, 'event_type': 'ARRIVE', 'stop_id': 'TI', 'time': 2492.2, 'coordinates': {'lat': 33.764475, 'lng': -118.376411}}, {'id': 7, 'event_type': 'DEPART', 'stop_id': 'TI', 'time': 2577.8999999999996, 'coordinates': {'lat': 33.764475, 'lng': -118.376411}}, {'id': 8, 'event_type': 'ARRIVE', 'stop_id': 'JX', 'time': 2644.3999999999996, 'coordinates': {'lat': 33.764621, 'lng': -118

In [13]:
for key, value in drone_naive_paths.items():
    print({key:value})

{'LA': [{'id': 1, 'stop': 'BO', 'coordinates': {'lat': 33.918699, 'lng': -118.324843}}, {'id': 2, 'stop': 'DR', 'coordinates': {'lat': 33.762769, 'lng': -118.375247}}, {'id': 3, 'stop': 'WR', 'coordinates': {'lat': 33.763597, 'lng': -118.375584}}, {'id': 4, 'stop': 'TI', 'coordinates': {'lat': 33.764475, 'lng': -118.376411}}, {'id': 5, 'stop': 'JX', 'coordinates': {'lat': 33.764621, 'lng': -118.37662}}, {'id': 6, 'stop': 'LD', 'coordinates': {'lat': 33.762374, 'lng': -118.375609}}, {'id': 7, 'stop': 'YK', 'coordinates': {'lat': 33.762304, 'lng': -118.376048}}, {'id': 8, 'stop': 'ZP', 'coordinates': {'lat': 33.763464, 'lng': -118.376728}}, {'id': 9, 'stop': 'IM', 'coordinates': {'lat': 33.763868, 'lng': -118.377216}}, {'id': 10, 'stop': 'SK', 'coordinates': {'lat': 33.764444, 'lng': -118.378505}}, {'id': 11, 'stop': 'SG', 'coordinates': {'lat': 33.764515, 'lng': -118.379196}}, {'id': 12, 'stop': 'ML', 'coordinates': {'lat': 33.76409, 'lng': -118.38009}}, {'id': 13, 'stop': 'AT', 'coordi

In [14]:
def process_single_city_events(events):
    """Process the event data for a single city and return a DataFrame with structured data."""
    # Check if events list is empty
    if not events:
        print("No events to process.")
        return pd.DataFrame()

    # Normalize data and create DataFrame for each event list independently
    df = pd.DataFrame(events)

    # Ensure that all expected columns are present
    expected_keys = {'id', 'coordinates', 'time', 'event_type'}
    if not expected_keys.issubset(df.columns):
        missing = expected_keys - set(df.columns)
        raise ValueError(f"Missing keys in data: {missing}")

    # Calculate and handle each attribute, ensuring all entries are processed independently
    processed_df = pd.DataFrame({
        'id': df['id'],
        'lat': df['coordinates'].apply(lambda x: x.get('lat', 0)),  # Default to 0 if no latitude
        'lon': df['coordinates'].apply(lambda x: x.get('lng', 0)),  # Default to 0 if no longitude
        'elevMeters': 0,
        'objectID': df['id'],
        'startTimeSec': df['time'],
        'nodeName': df.apply(lambda x: f"Node {x['id']}", axis=1),
        'nodeType': 'Waypoint',
        'popupText': df.apply(lambda x: f"{x['id']} Event: {x['event_type']}<br>Time: {x['time']}", axis=1),
        'leafletIconPrefix': 'fa',
        'leafletIconType': df['id'].apply(lambda x: 'flag' if x % 2 == 0 else 'circle'),
        'leafletColor': df['id'].apply(lambda x: 'green' if x % 2 == 0 else 'blue'),
        'leafletIconText': df['id'].apply(str),
        'cesiumIconType': 'pin',
        'cesiumColor': df['id'].apply(lambda x: 'red' if x % 2 == 0 else 'yellow'),
        'cesiumIconText': df['id'].apply(str),
        'altMeters': 0,  # Assuming altitude is not varying
        'stopID': df['stop_id']
    })

    return processed_df

# Dictionary to hold processed DataFrames for each city
drone_naive_events_dataframes = {}

# Loop over each city and its associated events data
for city, events in drone_naive_events.items():
    drone_naive_events_dataframes[city] = process_single_city_events(events)

In [15]:
drone_naive_nodes = drone_naive_events_dataframes
drone_naive_nodes

{'LA':       id        lat         lon  elevMeters  objectID  startTimeSec  nodeName  \
 0      1  33.918699 -118.324843           0         1           0.0    Node 1   
 1      2  33.762769 -118.375247           0         2        2288.5    Node 2   
 2      3  33.762769 -118.375247           0         3        2356.0    Node 3   
 3      4  33.763597 -118.375584           0         4        2369.7    Node 4   
 4      5  33.763597 -118.375584           0         5        2475.7    Node 5   
 ..   ...        ...         ...         ...       ...           ...       ...   
 323  324  33.766300 -118.375508           0       324       21802.3  Node 324   
 324  325  33.766300 -118.375508           0       325       21881.3  Node 325   
 325  326  33.766809 -118.377060           0       326       21970.9  Node 326   
 326  327  33.766809 -118.377060           0       327       22032.9  Node 327   
 327  328  33.918699 -118.324843           0       328       24539.9  Node 328   
 
      no

In [16]:
drone_naive_events

{'LA': [{'id': 1,
   'event_type': 'DEPART',
   'stop_id': 'BO',
   'time': 0,
   'coordinates': {'lat': 33.918699, 'lng': -118.324843}},
  {'id': 2,
   'event_type': 'ARRIVE',
   'stop_id': 'DR',
   'time': 2288.5,
   'coordinates': {'lat': 33.762769, 'lng': -118.375247}},
  {'id': 3,
   'event_type': 'DEPART',
   'stop_id': 'DR',
   'time': 2356.0,
   'coordinates': {'lat': 33.762769, 'lng': -118.375247}},
  {'id': 4,
   'event_type': 'ARRIVE',
   'stop_id': 'WR',
   'time': 2369.7,
   'coordinates': {'lat': 33.763597, 'lng': -118.375584}},
  {'id': 5,
   'event_type': 'DEPART',
   'stop_id': 'WR',
   'time': 2475.7,
   'coordinates': {'lat': 33.763597, 'lng': -118.375584}},
  {'id': 6,
   'event_type': 'ARRIVE',
   'stop_id': 'TI',
   'time': 2492.2,
   'coordinates': {'lat': 33.764475, 'lng': -118.376411}},
  {'id': 7,
   'event_type': 'DEPART',
   'stop_id': 'TI',
   'time': 2577.8999999999996,
   'coordinates': {'lat': 33.764475, 'lng': -118.376411}},
  {'id': 8,
   'event_type':

In [17]:
vehicle_naive_events

{'LA': [{'id': 1,
   'event_type': 'DEPART',
   'stop_id': 'BO',
   'time': 0,
   'coordinates': {'lat': 33.918699, 'lng': -118.324843}},
  {'id': 2,
   'event_type': 'ARRIVE',
   'stop_id': 'DR',
   'time': 2288.5,
   'coordinates': {'lat': 33.762769, 'lng': -118.375247}},
  {'id': 3,
   'event_type': 'SERVICE_START',
   'stop_id': 'DR',
   'time': 2288.5,
   'coordinates': {'lat': 33.762769, 'lng': -118.375247}},
  {'id': 4,
   'event_type': 'SERVICE_END',
   'stop_id': 'DR',
   'time': 2356.0,
   'coordinates': {'lat': 33.762769, 'lng': -118.375247}},
  {'id': 5,
   'event_type': 'DEPART',
   'stop_id': 'DR',
   'time': 2356.0,
   'coordinates': {'lat': 33.762769, 'lng': -118.375247}},
  {'id': 6,
   'event_type': 'ARRIVE',
   'stop_id': 'WR',
   'time': 2369.7,
   'coordinates': {'lat': 33.763597, 'lng': -118.375584}},
  {'id': 7,
   'event_type': 'SERVICE_START',
   'stop_id': 'WR',
   'time': 2369.7,
   'coordinates': {'lat': 33.763597, 'lng': -118.375584}},
  {'id': 8,
   'event

In [18]:
# Dictionary to hold processed DataFrames for each city
vehicle_naive_events_dataframes = {}

# Loop over each city and its associated events data
for city, events in drone_naive_events.items():
    vehicle_naive_events_dataframes[city] = process_single_city_events(events)

In [19]:
vehicle_naive_nodes = vehicle_naive_events_dataframes

In [21]:
drone_mp = {}
for city, df in drone_naive_nodes.items():
    drone_mp[city] = {}
    for i, row in df.iterrows():
        drone_mp[city][row['stopID']] = i

In [22]:
vehicle_mp = {}
for city, df in vehicle_naive_nodes.items():
    vehicle_mp[city] = {}
    for i, row in df.iterrows():
        vehicle_mp[city][row['stopID']] = i

In [23]:
drone_props = {
    'Austin': {'modelFile': 'veroviz/models/drone.gltf', 'color': 'orange'},
    'Chicago': {'modelFile': 'veroviz/models/drone.gltf', 'color': 'pink'},
    'Seattle': {'modelFile': 'veroviz/models/drone.gltf', 'color': 'black'},
    'Boston': {'modelFile': 'veroviz/models/drone.gltf', 'color': 'red'},
    'LA': {'modelFile': 'veroviz/models/drone.gltf', 'color': 'gray'}
}

def create_drone_only_rows(odID, objectID, drone_props, route, times):
    if not route:
        print(f"Empty drone received for {objectID}.")
        return []

    rows = []
    for i in range(len(route) - 1):
        start_point = route[i]
        end_point = route[i+1]
        row = {
            'odID': odID + i,
            'objectID': objectID,
            'modelFile': drone_props[objectID]['modelFile'],
            'startLat': start_point['lat'],
            'startLon': start_point['lng'],
            'endLat': end_point['lat'],
            'endLon': end_point['lng'],
            'startElevMeters': 0,  # Example default value
            'endElevMeters': 0,
            'startAltMeters': 100,
            'endAltMeters': 100,
            'startTimeSec': times[i],
            'endTimeSec': times[i+1],
            'leafletColor': drone_props[objectID]['color'],
            'leafletWeight': 5,
            'leafletStyle': 'solid',
            'leafletOpacity': 0.8,
            'leafletCurveType': 'straight',
            'leafletCurvature': 0,
            'arcCurveType': 'straight',
            'useArrows': True,
            'cesiumColor': drone_props[objectID]['color'],
            'cesiumWeight': 5,
            'cesiumStyle': 'solid',
            'cesiumOpacity': 0.8,
            'popupText': f'Route segment {i+1} for {objectID}'
        }
        rows.append(row)

    return rows

def process_drone_paths(drone_paths, drone_props, drone_events, mp):
    all_rows = []
    odID = 1

    for region, paths in drone_paths.items():
        print(f"Processing region: {region}")

        event_type_by_stop = {event['stop_id']: event['event_type'] for event in drone_events[region]}

        for i in range(len(paths) - 1):
            start_stop = paths[i]['stop']
            end_stop = paths[i+1]['stop']

            start = paths[i]['coordinates']
            end = paths[i+1]['coordinates']

            start_event = event_type_by_stop.get(start_stop, '')
            end_event = event_type_by_stop.get(end_stop, '')

            if 'LAUNCH' in start_event or 'LAND' in start_event or 'LAUNCH' in end_event or 'LAND' in end_event:
                print(f"Using straight-line path for segment {i+1} between {start_stop} and {end_stop}")
                route = [{'lat': start['lat'], 'lng': start['lng']}, {'lat': end['lat'], 'lng': end['lng']}]
            else:
                print(f"Using OSRM route for segment {i+1} between {start_stop} and {end_stop}")
                route, error = get_osrm_route((start['lat'], start['lng']), (end['lat'], end['lng']))
                if error:
                    print(f"Error processing segment {i+1} in {region}: {error}")
                    continue  # Skip this segment due to error

            # interpolate times
            times = []
            start_time = vehicle_only_nodes[region].loc[mp[region][paths[i]['stop']], 'startTimeSec']
            end_time = vehicle_only_nodes[region].loc[mp[region][paths[i+1]['stop']], 'startTimeSec']
            if route:
                tot = 0
                for i in range(len(route) - 1):
                    di = geopy.distance.distance((start['lat'], start['lng']), (end['lat'], end['lng'])).km
                    tot += di
                times.append(start_time)
                for i in range(len(route) - 1):
                    di = geopy.distance.distance((start['lat'], start['lng']), (end['lat'], end['lng'])).km
                    times.append(times[-1] if tot == 0 else times[-1] + (end_time - start_time) * di / tot)

            rows = create_drone_only_rows(odID, region, drone_props, route, times)
            all_rows.extend(rows)
            odID += len(rows)

    return all_rows





In [24]:
drone_naive_assignments = process_drone_paths(drone_naive_paths, drone_props, drone_naive_events, drone_mp)

Processing region: LA
Using OSRM route for segment 1 between BO and DR
Using OSRM route for segment 2 between DR and WR
Using OSRM route for segment 3 between WR and TI
Using straight-line path for segment 4 between TI and JX
Using straight-line path for segment 5 between JX and LD
Using OSRM route for segment 6 between LD and YK
Using straight-line path for segment 7 between YK and ZP
Using straight-line path for segment 8 between ZP and IM
Using OSRM route for segment 9 between IM and SK
Using straight-line path for segment 10 between SK and SG
Using straight-line path for segment 11 between SG and ML
Using OSRM route for segment 12 between ML and AT
Using OSRM route for segment 13 between AT and UQ
Using OSRM route for segment 14 between UQ and RU
Using straight-line path for segment 15 between RU and JJ
Using straight-line path for segment 16 between JJ and YD
Using OSRM route for segment 17 between YD and IF
Using OSRM route for segment 18 between IF and WW
Using straight-line pat

In [20]:
# Dictionary to hold processed DataFrames for each city
vehicle_naive_events_dataframes = {}

# Loop over each city and its associated events data
for city, events in vehicle_naive_events.items():
    vehicle_naive_events_dataframes[city] = process_single_city_events(events)

In [21]:
vehicle_naive_nodes = vehicle_naive_events_dataframes
vehicle_naive_nodes

{'LA':       id        lat         lon  elevMeters  objectID  startTimeSec  nodeName  \
 0      1  33.918699 -118.324843           0         1           0.0    Node 1   
 1      2  33.762769 -118.375247           0         2        2288.5    Node 2   
 2      3  33.762769 -118.375247           0         3        2288.5    Node 3   
 3      4  33.762769 -118.375247           0         4        2356.0    Node 4   
 4      5  33.762769 -118.375247           0         5        2356.0    Node 5   
 ..   ...        ...         ...         ...       ...           ...       ...   
 493  494  33.766809 -118.377060           0       494       21970.9  Node 494   
 494  495  33.766809 -118.377060           0       495       21970.9  Node 495   
 495  496  33.766809 -118.377060           0       496       22032.9  Node 496   
 496  497  33.766809 -118.377060           0       497       22032.9  Node 497   
 497  498  33.918699 -118.324843           0       498       24539.9  Node 498   
 
      no

In [27]:
naive_vehicle_only_rows = process_vehicle_paths(vehicle_naive_paths, vehicle_props, vehicle_mp)
vehicle_naive_assignments = naive_vehicle_only_rows

Processing path for LA with 113 stops.


In [28]:
vehicle_optimize_events_directory_path = '../algorithm/optimized_vehicle_events'
vehicle_optimize_paths_directory_path = '../algorithm/optimized_vehicle_paths'
drone_optimize_events_directory_path = '../algorithm/optimized_drone_events'
drone_optimize_paths_directory_path = '../algorithm/optimized_drone_paths'

def load_json_files(directory):
    data_objects = {}
    for filename in os.listdir(directory):
        if filename.endswith('.json'):  # Ensure it's a JSON file
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r') as file:
                data_objects[filename] = json.load(file)
    return data_objects

vehicle_optimize_events = load_json_files(vehicle_optimize_events_directory_path)
vehicle_optimize_paths = load_json_files(vehicle_optimize_paths_directory_path)
drone_optimize_events = load_json_files(drone_optimize_events_directory_path)
drone_optimize_paths = load_json_files(drone_optimize_paths_directory_path)

In [39]:
vehicle_naive_assignments = pd.DataFrame(vehicle_naive_assignments)
drone_naive_assignments = pd.DataFrame(drone_naive_assignments)

In [42]:
drone_naive_assignments['odID'] += len(vehicle_naive_assignments)

In [54]:
assignments = pd.concat([vehicle_naive_assignments, drone_naive_assignments], ignore_index=True)
assignments['modelScale'] = 100
assignments['modelMinPxSize'] = 75
assignments['ganttColor'] = 'darkgray'
assignments['wayname'] = None
assignments['waycategory'] = None
assignments['surface'] = None
assignments['waytype'] = None
assignments['steepness'] = None
assignments['tollway'] = None

In [45]:
assignments

,odID,objectID,modelFile,startLat,startLon,endLat,endLon,startElevMeters,endElevMeters,startAltMeters,...,popupText,modelScale,modelMinPxSize,ganttColor,wayname,waycategory,surface,waytype,steepness,tollway
0,1,LA,veroviz/models/ub_truck.gltf,33.91870,-118.32644,33.91878,-118.32644,0,0,0,...,Route segment 1 for LA,100,75,darkgray,None,None,None,None,None,None
1,2,LA,veroviz/models/ub_truck.gltf,33.91878,-118.32644,33.91915,-118.32645,0,0,0,...,Route segment 2 for LA,100,75,darkgray,None,None,None,None,None,None
2,3,LA,veroviz/models/ub_truck.gltf,33.91915,-118.32645,33.91928,-118.32644,0,0,0,...,Route segment 3 for LA,100,75,darkgray,None,None,None,None,None,None
3,4,LA,veroviz/models/ub_truck.gltf,33.91928,-118.32644,33.91937,-118.32645,0,0,0,...,Route segment 4 for LA,100,75,darkgray,None,None,None,None,None,None
4,5,LA,veroviz/models/ub_truck.gltf,33.91937,-118.32645,33.91943,-118.32645,0,0,0,...,Route segment 5 for LA,100,75,darkgray,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2105,4655,LA,veroviz/models/drone.gltf,33.91710,-118.32645,33.91739,-118.32644,0,0,100,...,Route segment 615 for LA,100,75,darkgray,None,None,None,None,None,None
2106,4656,LA,veroviz/models/drone.gltf,33.91739,-118.32644,33.91760,-118.32644,0,0,100,...,Route segment 616 for LA,100,75,darkgray,None,None,None,None,None,None
2107,4657,LA,veroviz/models/drone.gltf,33.91760,-118.32644,33.91773,-118.32644,0,0,100,...,Route segment 617 for LA,100,75,darkgray,None,None,None,None,None,None
2108,4658,LA,veroviz/models/drone.gltf,33.91773,-118.32644,33.91801,-118.32644,0,0,100,...,Route segment 618 for LA,100,75,darkgray,None,None,None,None,None,None


In [46]:
drone_naive_nodes['LA']['objectID'] += len(vehicle_naive_nodes['LA'])
drone_naive_nodes['LA']['id'] += len(vehicle_naive_nodes['LA'])

In [55]:
nodes_df = pd.concat([vehicle_naive_nodes['LA'], drone_naive_nodes['LA']], ignore_index=True)

In [48]:
nodes_df

,id,lat,lon,elevMeters,objectID,startTimeSec,nodeName,nodeType,popupText,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText,altMeters,stopID
0,1,33.918699,-118.324843,0,1,0.0,Node 1,Waypoint,1 Event: DEPART<br>Time: 0.0,fa,circle,blue,1,pin,yellow,1,0,BO
1,2,33.762769,-118.375247,0,2,2288.5,Node 2,Waypoint,2 Event: ARRIVE<br>Time: 2288.5,fa,flag,green,2,pin,red,2,0,DR
2,3,33.762769,-118.375247,0,3,2356.0,Node 3,Waypoint,3 Event: DEPART<br>Time: 2356.0,fa,circle,blue,3,pin,yellow,3,0,DR
3,4,33.763597,-118.375584,0,4,2369.7,Node 4,Waypoint,4 Event: ARRIVE<br>Time: 2369.7,fa,flag,green,4,pin,red,4,0,WR
4,5,33.763597,-118.375584,0,5,2475.7,Node 5,Waypoint,5 Event: DEPART<br>Time: 2475.7,fa,circle,blue,5,pin,yellow,5,0,WR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,652,33.766300,-118.375508,0,652,21802.3,Node 324,Waypoint,324 Event: ARRIVE<br>Time: 21802.29999999998,fa,flag,green,324,pin,red,324,0,QG
324,653,33.766300,-118.375508,0,653,21881.3,Node 325,Waypoint,325 Event: DEPART<br>Time: 21881.29999999998,fa,circle,blue,325,pin,yellow,325,0,QG
325,654,33.766809,-118.377060,0,654,21970.9,Node 326,Waypoint,326 Event: ARRIVE<br>Time: 21970.89999999998,fa,flag,green,326,pin,red,326,0,QI
326,655,33.766809,-118.377060,0,655,22032.9,Node 327,Waypoint,327 Event: DEPART<br>Time: 22032.89999999998,fa,circle,blue,327,pin,yellow,327,0,QI


In [57]:
assignments

,odID,objectID,modelFile,startLat,startLon,endLat,endLon,startElevMeters,endElevMeters,startAltMeters,...,popupText,modelScale,modelMinPxSize,ganttColor,wayname,waycategory,surface,waytype,steepness,tollway
0,1,LA,/veroviz/models/ub_truck.gltf,33.91870,-118.32644,33.91878,-118.32644,0,0,0,...,Route segment 1 for LA,100,75,darkgray,None,None,None,None,None,None
1,2,LA,/veroviz/models/ub_truck.gltf,33.91878,-118.32644,33.91915,-118.32645,0,0,0,...,Route segment 2 for LA,100,75,darkgray,None,None,None,None,None,None
2,3,LA,/veroviz/models/ub_truck.gltf,33.91915,-118.32645,33.91928,-118.32644,0,0,0,...,Route segment 3 for LA,100,75,darkgray,None,None,None,None,None,None
3,4,LA,/veroviz/models/ub_truck.gltf,33.91928,-118.32644,33.91937,-118.32645,0,0,0,...,Route segment 4 for LA,100,75,darkgray,None,None,None,None,None,None
4,5,LA,/veroviz/models/ub_truck.gltf,33.91937,-118.32645,33.91943,-118.32645,0,0,0,...,Route segment 5 for LA,100,75,darkgray,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4654,4655,LA,/veroviz/models/drone.gltf,33.91710,-118.32645,33.91739,-118.32644,0,0,100,...,Route segment 615 for LA,100,75,darkgray,None,None,None,None,None,None
4655,4656,LA,/veroviz/models/drone.gltf,33.91739,-118.32644,33.91760,-118.32644,0,0,100,...,Route segment 616 for LA,100,75,darkgray,None,None,None,None,None,None
4656,4657,LA,/veroviz/models/drone.gltf,33.91760,-118.32644,33.91773,-118.32644,0,0,100,...,Route segment 617 for LA,100,75,darkgray,None,None,None,None,None,None
4657,4658,LA,/veroviz/models/drone.gltf,33.91773,-118.32644,33.91801,-118.32644,0,0,100,...,Route segment 618 for LA,100,75,darkgray,None,None,None,None,None,None


In [52]:
assignments = vehicle_naive_assignments
assignments['modelScale'] = 100
assignments['modelMinPxSize'] = 75
assignments['ganttColor'] = 'darkgray'
assignments['wayname'] = None
assignments['waycategory'] = None
assignments['surface'] = None
assignments['waytype'] = None
assignments['steepness'] = None
assignments['tollway'] = None
nodes_df = vehicle_naive_nodes['LA']
vrv.createCesium(assignments=assignments,
                 nodes=nodes_df,
                 cesiumDir=os.getcwd()+'/Cesium-1.116',
                 problemDir='veroviz/la')

In [58]:
vrv.createCesium(assignments=assignments,
                 nodes=nodes_df,
                 cesiumDir=os.getcwd()+'/Cesium-1.116',
                 problemDir='veroviz/la')

/home/max/miniconda3/envs/vrpd/lib/python3.12/site-packages/veroviz/createCesium.py:768: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  path = path.append({
/home/max/miniconda3/envs/vrpd/lib/python3.12/site-packages/veroviz/createCesium.py:768: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  path = path.append({
/home/max/miniconda3/envs/vrpd/lib/python3.12/site-packages/veroviz/createCesium.py:768: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  path = path.append({
/home/max/miniconda3/envs/vrpd/lib/python3.12/site-packages/veroviz/createCesium.py:768: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  path = path.append({
/home/max/miniconda3

Message: File selector was written to /home/max/vrpd-amazon-dataset/visual/Cesium-1.116/veroviz/la/;veroviz;la.vrv ...
Message: Configs were written to /home/max/vrpd-amazon-dataset/visual/Cesium-1.116/veroviz/la/config.js ...
Message: Nodes were written to /home/max/vrpd-amazon-dataset/visual/Cesium-1.116/veroviz/la/displayNodes.js ...
Message: Assignments (.js) were written to /home/max/vrpd-amazon-dataset/visual/Cesium-1.116/veroviz/la/displayPaths.js ...
Message: Assignments (.czml) were written to /home/max/vrpd-amazon-dataset/visual/Cesium-1.116/veroviz/la/routes.czml ...
